Для стандартного датасету

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

використати різні гіперпараметри алгоритму SVC і порівняти результати

*   первинний аналіз даних (відстуність пропусків, наявність категоріальних фіч, duplicated, Nan) (+)
*   фича інжиніринг (побудувати 1-2 нові фічі) (+)
*   Убрать кореляцію в фічах (+)
*   поділ датасету на train, validate, test (+ random_state) (+)
*   GridSearchCV (+)
*   Scaling (+)
*   тренування базової моделі із дефолтними гіперпараметрами (кожну модель) (+)
*   підбір гіперпараметрів (кожну модель) (+)
*   побудувати модель різними способами (+)
*   Metrics (+)
*   оцінка результатів (порівняння всіх на тестовій частині, описати яка краще) (+)
*  Графічний аналіз, (якщо встигну)

## INIT BLOCK

### Import Requirements

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.metrics import (r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report,
                             precision_score, recall_score, f1_score, )
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.preprocessing import (StandardScaler, RobustScaler, PowerTransformer)
from sklearn.svm import (SVC, NuSVC, LinearSVC)
from sklearn.utils import Bunch


### Init

In [ ]:
RANDOM_STATE: int = 1729

if __name__ == "__main__":
    digits_bunch: Bunch = load_digits(as_frame=True)
    digits_dataframe: pd.DataFrame = digits_bunch.frame

    working_dataframe: pd.DataFrame = digits_dataframe.copy()

    target: str = "target"


# Data analysis, clining and optomization

### DataSet Description

In [ ]:
print(digits_bunch.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 1797
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

### DataFrame info

In [ ]:
working_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 65 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pixel_0_0  1797 non-null   float64
 1   pixel_0_1  1797 non-null   float64
 2   pixel_0_2  1797 non-null   float64
 3   pixel_0_3  1797 non-null   float64
 4   pixel_0_4  1797 non-null   float64
 5   pixel_0_5  1797 non-null   float64
 6   pixel_0_6  1797 non-null   float64
 7   pixel_0_7  1797 non-null   float64
 8   pixel_1_0  1797 non-null   float64
 9   pixel_1_1  1797 non-null   float64
 10  pixel_1_2  1797 non-null   float64
 11  pixel_1_3  1797 non-null   float64
 12  pixel_1_4  1797 non-null   float64
 13  pixel_1_5  1797 non-null   float64
 14  pixel_1_6  1797 non-null   float64
 15  pixel_1_7  1797 non-null   float64
 16  pixel_2_0  1797 non-null   float64
 17  pixel_2_1  1797 non-null   float64
 18  pixel_2_2  1797 non-null   float64
 19  pixel_2_3  1797 non-null   float64
 20  pixel_2_

### DataFrame head

In [ ]:
working_dataframe.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


### Перевіряємо на дублікати

In [ ]:
print(f"{'Є дублікати, потрібна обробка датасету.' if working_dataframe.duplicated().sum() else 'Дублікатів немає.'}", end="\n\n")

Дублікатів немає.



### Обробка відсутніх значеннь

In [ ]:
print(f"{'Є пропущені значення, потрібна обробка датасету.' if working_dataframe.isna().any().any() else 'Пропущених значень немає.'}", end="\n\n")

Пропущених значень немає.



### Генеруємо статистику

In [ ]:
working_dataframe.describe(include="all", percentiles=[.25, .5, .75])

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
count,1797.0,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,...,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000
mean,0.0,0.303840,5.204786,11.835838,11.848080,5.781859,1.362270,0.129661,0.005565,1.993879,...,0.206455,0.000556,0.279354,5.557596,12.089037,11.809126,6.764051,2.067891,0.364496,4.490818
std,0.0,0.907192,4.754826,4.248842,4.287388,5.666418,3.325775,1.037383,0.094222,3.196160,...,0.984401,0.023590,0.934302,5.103019,4.374694,4.933947,5.900623,4.090548,1.860122,2.865304
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,1.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,11.000000,10.000000,0.000000,0.000000,0.000000,2.000000
50%,0.0,0.000000,4.000000,13.000000,13.000000,4.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.000000,13.000000,14.000000,6.000000,0.000000,0.000000,4.000000
75%,0.0,0.000000,9.000000,15.000000,15.000000,11.000000,0.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,10.000000,16.000000,16.000000,12.000000,2.000000,0.000000,7.000000
max,0.0,8.000000,16.000000,16.000000,16.000000,16.000000,16.000000,15.000000,2.000000,16.000000,...,13.000000,1.000000,9.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,9.000000


# Feature Engineering

feature engineering по тз пропускаємо.

# Preparing data for training

### Розбиваємо датасет на фічі та таргет

In [ ]:
x_features: pd.DataFrame = working_dataframe.copy().drop(target, axis=1)
y_target: pd.DataFrame = working_dataframe[target]

### Масштабування фічів

In [ ]:
scaler: StandardScaler = StandardScaler().fit(X=x_features)  # 0.9898
# scaler: PowerTransformer = PowerTransformer().fit(X=x_features)  # 0.9847
# scaler: RobustScaler = RobustScaler().fit(X=x_features)  # 0.9847

x_scaled: np.ndarray = scaler.transform(X=x_features)

### Розбиваємо дані на тренувальну, валідаційну та тестову частину

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_target, test_size=0.33, random_state=RANDOM_STATE, stratify=y_target)
x_validate, x_test, y_validate, y_test = train_test_split(x_test, y_test, test_size=0.33, random_state=RANDOM_STATE, stratify=y_test)

print(x_train.shape)
print(x_validate.shape)
print(x_test.shape)

(1203, 64)
(397, 64)
(197, 64)


### Перевіряємо розподіл классів

In [ ]:
print(f"Загальний таргет:\n{working_dataframe[target].value_counts()}", end="\n\n")
print(f"Тренувальний таргет:\n{y_train.value_counts()}")

Загальний таргет:
3    183
1    182
5    182
4    181
6    181
9    180
7    179
0    178
2    177
8    174
Name: target, dtype: int64

Тренувальний таргет:
3    123
1    122
5    122
4    121
6    121
9    121
7    120
0    119
2    118
8    116
Name: target, dtype: int64


Класи майже збалансовані. Найбільша разниця становить ~ 6%. Вважаю, що балансування класів не потрібно.

### Балансування классів

In [ ]:
x_resampled, y_resampled = x_train, y_train

# FIT PREDICT

## Functions for fit models, make predict and generate metrics

In [ ]:
def print_model_name(name: str) -> None:
    print("'" * 60)
    print(f"MODEL: {name}")

In [ ]:
def value_results_main(predict, y_test):
    accuracy = accuracy_score(y_true=y_test, y_pred=predict)
    precision = precision_score(y_true=y_test, y_pred=predict, average="micro")
    recall = recall_score(y_true=y_test, y_pred=predict, average="micro")
    f1 = f1_score(y_true=y_test, y_pred=predict, average="micro")

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

In [ ]:
def value_regression(predict, y_test):
    model_r2: np.float64 = r2_score(y_true=y_test, y_pred=predict)
    model_mean_squared_error: np.float64 = mean_squared_error(y_true=y_test, y_pred=predict)
    model_mean_absolute_error: np.float64 = mean_absolute_error(y_true=y_test, y_pred=predict)

    print(f"R2:                         {model_r2}")
    print(f"Mean_squared_error (MSE):   {model_mean_squared_error}")
    print(f"Mean_absolute_error (MAE):  {model_mean_absolute_error}")

In [ ]:
def value_results(name:str, predict, y_test=y_test):
    print_model_name(name=name)
    try:
      value_results_main(predict=predict, y_test=y_test)
    except ValueError:
      value_regression(predict=predict, y_test=y_test)

    print("'" * 60)

In [ ]:
def train_model(model_class, name: str, y_train=y_resampled, y_validate=y_validate, grid_params=None):
    print_model_name(name=name)

    if grid_params:
        model: GridSearchCV = GridSearchCV(estimator=model_class, param_grid=grid_params).fit(X=x_resampled, y=y_train)

        print("Best params: ", model.best_params_)
        print("Best score: ", model.best_score_)

    else:
        model = model_class.fit(X=x_resampled, y=y_train)

    y_pred = model.predict(X=x_validate)

    try:
        print(f"Classification report:\n{classification_report(y_true=y_validate, y_pred=y_pred)}")
        value_results_main(predict=y_pred, y_test=y_validate)

    except ValueError:
        value_regression(predict=y_pred, y_test=y_validate)

    return model


## Тренуємо моделі

### SVC()

In [ ]:
## SVC() with default hyperparams
model_svc_default: SVC = train_model(model_class=SVC(), name="RandomForestClassifier with default hyperparameters")

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: RandomForestClassifier with default hyperparameters
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40
           2       1.00      1.00      1.00        39
           3       0.97      0.95      0.96        40
           4       1.00      0.97      0.99        40
           5       0.98      1.00      0.99        40
           6       1.00      1.00      1.00        40
           7       0.97      1.00      0.99        39
           8       0.97      1.00      0.99        39
           9       1.00      0.97      0.99        40

    accuracy                           0.99       397
   macro avg       0.99      0.99      0.99       397
weighted avg       0.99      0.99      0.99       397

Accuracy: 0.9899244332493703
Precision: 0.9899244332493703
Recall: 0.9899244332493703
F1 Score: 0.

In [ ]:
## SVC() з підбіром параметрів через GridSearchCV linear kernel
grid_params_svc_linear_kernel: dict = {
    "C": [1.5],  # float, default=1.0
    "kernel": ["linear"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "shrinking": [True],  # bool, default=True
    "probability": [True],  # bool, default=False
    "tol": [1e-3],  # float, default=1e-3
    "cache_size": [200],  # float, default=200
    "class_weight": [None],  # dict or ‘balanced’, default=None
    "verbose": [True],  # bool, default=False
    "max_iter": [-1],  # int, default=-1
    "decision_function_shape": ["ovr"],  # ["ovo", "ovr"], default="ovr"
    "break_ties": [True],  # bool, default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
}
grid_model_svc_linear_kernel = train_model(model_class=SVC(), name="SVC with Grid hyperparameters", grid_params=grid_params_svc_linear_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'C': 1.5, 'break_ties': True, 'cache_size': 200, 'class_weight': None, 'decision_function_shape': 'ovr', 'kernel': 'linear', 'max_iter': -1, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best score:  0.9742392807745505
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      0.97      0.99        40
           2       1.00      1.00      1.00        39
           3       1.00      0.95      0.97        40
           4       1.00      1.00      1.00        40
           5       0.98      1.00      0.99        40
           6       1.00      1.00      1.00        40
           7       1.00      0.97      0.99        39
           8       0.89      1.00      0.94        39
         

In [ ]:
# SVC() з підбіром параметрів через GridSearchCV poly kernel
grid_params_svc_poly_kernel: dict = {
    "C": [2],  # float, default=1.0
    "break_ties": [False],  # bool, default=False
    "cache_size": [200],  # float, default=200
    "class_weight": [None],  # dict or "balanced", default=None
    "coef0": [1],  # float, default=0.0   ‘poly’ and ‘sigmoid’
    "decision_function_shape": ["ovr"],  # ["ovo", "ovr"], default="ovr"
    "degree": [2],  # int, default=3 only for "poly" kernel
    "gamma": ["scale"],  # ["scale", "auto"] or float, default="scale"
    "kernel": ["poly"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "max_iter": [-1],  # int, default=-1
    "probability": [True],  # bool, default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
    "shrinking": [True],  # bool, default=True
    "tol": [1e-3],  # float, default=1e-3
    "verbose": [True],  # bool, default=False
}
grid_model_svc_poly_kernel = train_model(model_class=SVC(), name="SVC with Grid hyperparameters", grid_params=grid_params_svc_poly_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'C': 2, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 1, 'decision_function_shape': 'ovr', 'degree': 2, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': -1, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best score:  0.9858782849239281
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40
           2       1.00      1.00      1.00        39
           3       1.00      1.00      1.00        40
           4       1.00      0.97      0.99        40
           5       0.98      1.00      0.99        40
           6       1.00      1.00      1.00        40
           7       0.97      0.97      0.97        39
           8       0.97

In [ ]:
# SVC() з підбіром параметрів через GridSearchCV rbf kernel
grid_params_svc_rbf_kernel: dict = {
    "C": [3],  # float, default=1.0
    "break_ties": [True],  # bool, default=False
    "cache_size": [200],  # float, default=200
    "class_weight": [None],  # dict or "balanced", default=None
    "decision_function_shape": ["ovr"],  # ["ovo", "ovr"], default="ovr"
    "gamma": ["auto"],  # ["scale", "auto"] or float, default="scale"
    "kernel": ["rbf"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "max_iter": [-1],  # int, default=-1
    "probability": [True],  # bool, default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
    "shrinking": [True],  # bool, default=True
    "tol": [0.15],  # float, default=1e-3
    "verbose": [True],  # bool, default=False
}
grid_model_svc_rbf_kernel = train_model(model_class=SVC(), name="SVC with Grid hyperparameters", grid_params=grid_params_svc_rbf_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'C': 3, 'break_ties': True, 'cache_size': 200, 'class_weight': None, 'decision_function_shape': 'ovr', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.15, 'verbose': True}
Best score:  0.9758990318118949
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40
           2       1.00      1.00      1.00        39
           3       1.00      0.97      0.99        40
           4       1.00      0.97      0.99        40
           5       1.00      1.00      1.00        40
           6       1.00      1.00      1.00        40
           7       0.97      1.00      0.99        39
           8       0.95      1.00      0.97        3

In [ ]:
# SVC() з підбіром параметрів через GridSearchCV sigmoid kernel
grid_params_svc_sigmoid_kernel: dict = {
    "C": [0.5],  # float, default=1.0
    "break_ties": [False],  # [True, False], default=False
    "cache_size": [200],  # float, default=200
    "class_weight": ["balanced"],  # dict or "balanced", default=None
    "coef0": [0.0],  # float, default=0.0    ‘poly’ and ‘sigmoid’
    "decision_function_shape": ["ovo"],  # ["ovo", "ovr"], default="ovr"
    "gamma": ["auto"],  # ["scale", "auto"] or float, default="scale"
    "kernel": ["sigmoid"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "max_iter": [-1],  # int, default=-1
    "probability": [True],  # [True, False], default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
    "shrinking": [True],  # [True, False], default=True
    "tol": [1e-3],  # float, default=1e-3
    "verbose": [True],  # [True, False], default=False
}
grid_model_svc_sigmoid_kernel = train_model(model_class=SVC(), name="SVC with Grid hyperparameters", grid_params=grid_params_svc_sigmoid_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'C': 0.5, 'break_ties': False, 'cache_size': 200, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'sigmoid', 'max_iter': -1, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best score:  0.9434958506224067
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.89      0.85      0.87        40
           2       0.88      0.92      0.90        39
           3       0.97      0.88      0.92        40
           4       1.00      0.93      0.96        40
           5       0.98      1.00      0.99        40
           6       0.98      1.00      0.99        40
           7       0.91      1.00      0.95        39
           8       0.97 

### LinearSVC()

In [ ]:
# LinearSVC() with default hyperparams
model_linear_svc_default: LinearSVC = train_model(model_class=LinearSVC(), name="LinearSVC with default hyperparameters")

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: LinearSVC with default hyperparameters
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.95      0.90      0.92        40
           2       0.97      0.97      0.97        39
           3       0.92      0.90      0.91        40
           4       1.00      0.95      0.97        40
           5       0.95      0.95      0.95        40
           6       0.93      1.00      0.96        40
           7       0.93      0.97      0.95        39
           8       0.90      0.92      0.91        39
           9       0.95      0.93      0.94        40

    accuracy                           0.95       397
   macro avg       0.95      0.95      0.95       397
weighted avg       0.95      0.95      0.95       397

Accuracy: 0.9496221662468514
Precision: 0.9496221662468514
Recall: 0.9496221662468514
F1 Score: 0.9496221662468

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# LinearSVC() з підбіром параметрів через GridSearchCV
grid_params_model_linear_svc: dict = {
    "C": [1],  # float, default=1.0
    "intercept_scaling": [1.0],  # float, default=1.0
    "max_iter": [1000],  # int, default=1000
    "verbose": [0],  # int, default=0
    "tol": [1e-4],  # float, default=1e-4
    "dual": [False],  # “auto” or bool, default=True
    "class_weight": ["balanced"],  # dict or [None, "balanced"], default=None
    "fit_intercept": [True],  # bool, default=True
    "loss": ["squared_hinge"],  # ["hinge", "squared_hinge"], default="squared_hinge"
    "multi_class": ["crammer_singer"],  # {"ovr", "crammer_singer"}, default="ovr"
    "penalty": ["l1"],  # ["l1", "l2"], default="l2"
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
}
grid_model_linear_svc = train_model(model_class=LinearSVC(), name="LinearSVC with Grid hyperparameters", grid_params=grid_params_model_linear_svc)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: LinearSVC with Grid hyperparameters


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best params:  {'C': 1, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1.0, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'crammer_singer', 'penalty': 'l1', 'random_state': 1729, 'tol': 0.0001, 'verbose': 0}
Best score:  0.9451417704011066
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.93      0.95      0.94        40
           2       0.93      0.97      0.95        39
           3       1.00      0.85      0.92        40
           4       1.00      0.97      0.99        40
           5       0.97      0.95      0.96        40
           6       0.97      0.97      0.97        40
           7       0.93      1.00      0.96        39
           8       0.86      0.95      0.90        39
           9       0.92      0.88      0.90        40

    accuracy                           0.95       397
   macro avg       0.95      0.95   

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### NuSVC()

In [ ]:
# NuSVC() with default hyperparams
model_nu_svc_default: NuSVC = train_model(model_class=NuSVC(), name="NuSVC with default hyperparameters")

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: NuSVC with default hyperparameters
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.98      1.00      0.99        40
           2       0.97      1.00      0.99        39
           3       0.97      0.90      0.94        40
           4       0.97      0.95      0.96        40
           5       0.98      1.00      0.99        40
           6       0.97      0.97      0.97        40
           7       0.91      1.00      0.95        39
           8       1.00      0.92      0.96        39
           9       0.95      0.95      0.95        40

    accuracy                           0.97       397
   macro avg       0.97      0.97      0.97       397
weighted avg       0.97      0.97      0.97       397

Accuracy: 0.9697732997481109
Precision: 0.9697732997481109
Recall: 0.9697732997481109
F1 Score: 0.9697732997481109


In [ ]:
# NuSVC() з підбіром параметрів через GridSearchCV linear kernel
grid_params_nu_svc_linear_kernel: dict = {
    "nu": [0.2],  # default=0.5
    "kernel": ["linear"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "shrinking": [True],  # bool, default=True
    "probability": [True],  # bool, default=False
    "tol": [1e-3],  # float, default=1e-3
    "cache_size": [200],  # float, default=200
    "class_weight": [None],  # dict or ‘balanced’, default=None
    "verbose": [True],  # bool, default=False
    "max_iter": [-1],  # int, default=-1
    "decision_function_shape": ["ovr"],  # ["ovo", "ovr"], default="ovr"
    "break_ties": [True],  # bool, default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
}
grid_model_nu_svc_linear_kernel = train_model(model_class=NuSVC(), name="NuSVC with Grid hyperparameters", grid_params=grid_params_nu_svc_linear_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: NuSVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'break_ties': True, 'cache_size': 200, 'class_weight': None, 'decision_function_shape': 'ovr', 'kernel': 'linear', 'max_iter': -1, 'nu': 0.2, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best score:  0.9684232365145228
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40
           2       1.00      1.00      1.00        39
           3       1.00      0.90      0.95        40
           4       1.00      0.95      0.97        40
           5       0.98      1.00      0.99        40
           6       0.98      1.00      0.99        40
           7       0.93      1.00      0.96        39
           8       0.95      0.95      0.95        39
      

In [ ]:
# NuSVC() з підбіром параметрів через GridSearchCV poly kernel
grid_params_nu_svc_poly_kernel: dict = {
    "nu": [0.001],  # default=0.5
    "break_ties": [False],  # bool, default=False
    "cache_size": [200],  # float, default=200
    "class_weight": [None],  # dict or "balanced", default=None
    "coef0": [1],  # float, default=0.0   ‘poly’ and ‘sigmoid’
    "decision_function_shape": ["ovr"],  # ["ovo", "ovr"], default="ovr"
    "degree": [2],  # int, default=3 only for "poly" kernel
    "gamma": ["scale"],  # ["scale", "auto"] or float, default="scale"
    "kernel": ["poly"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "max_iter": [-1],  # int, default=-1
    "probability": [True],  # bool, default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
    "shrinking": [True],  # bool, default=True
    "tol": [1e-3],  # float, default=1e-3
    "verbose": [True],  # bool, default=False
}
grid_model_nu_svc_poly_kernel = train_model(model_class=NuSVC(), name="NuSVC with Grid hyperparameters", grid_params=grid_params_nu_svc_poly_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: NuSVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 1, 'decision_function_shape': 'ovr', 'degree': 2, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': -1, 'nu': 0.001, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best score:  0.9867081604426003
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      0.97      0.99        40
           2       1.00      1.00      1.00        39
           3       1.00      1.00      1.00        40
           4       1.00      0.97      0.99        40
           5       0.98      1.00      0.99        40
           6       1.00      1.00      1.00        40
           7       0.97      0.97      0.97        39
           8    

In [ ]:
# NuSVC() з підбіром параметрів через GridSearchCV rbf kernel
grid_params_nu_svc_rbf_kernel: dict = {
    "nu": [0.001],  # default=0.5
    "break_ties": [True],  # bool, default=False
    "cache_size": [200],  # float, default=200
    "class_weight": [None],  # dict or "balanced", default=None
    "decision_function_shape": ["ovr"],  # ["ovo", "ovr"], default="ovr"
    "gamma": ["auto"],  # ["scale", "auto"] or float, default="scale"
    "kernel": ["rbf"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "max_iter": [-1],  # int, default=-1
    "probability": [True],  # bool, default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
    "shrinking": [True],  # bool, default=True
    "tol": [0.0015],  # float, default=1e-3
    "verbose": [True],  # bool, default=False
}
grid_model_nu_svc_rbf_kernel = train_model(model_class=NuSVC(), name="NuSVC with Grid hyperparameters", grid_params=grid_params_nu_svc_rbf_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: NuSVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'break_ties': True, 'cache_size': 200, 'class_weight': None, 'decision_function_shape': 'ovr', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1, 'nu': 0.001, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.0015, 'verbose': True}
Best score:  0.9734059474412172
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40
           2       1.00      1.00      1.00        39
           3       1.00      0.97      0.99        40
           4       1.00      0.97      0.99        40
           5       1.00      1.00      1.00        40
           6       1.00      1.00      1.00        40
           7       0.97      1.00      0.99        39
           8       0.95      1.00      0.97

In [ ]:
# NuSVC() з підбіром параметрів через GridSearchCV sigmoid kernel
grid_params_nu_svc_sigmoid_kernel: dict = {
    "nu": [0.6],  # default=0.5
    "break_ties": [False],  # [True, False], default=False
    "cache_size": [200],  # float, default=200
    "class_weight": ["balanced"],  # dict or "balanced", default=None
    "coef0": [0.0],  # float, default=0.0    ‘poly’ and ‘sigmoid’
    "decision_function_shape": ["ovo"],  # ["ovo", "ovr"], default="ovr"
    "gamma": ["auto"],  # ["scale", "auto"] or float, default="scale"
    "kernel": ["sigmoid"],  # ["linear", "poly", "rbf", "sigmoid", "precomputed"] or callable, default="rbf"
    "max_iter": [-1],  # int, default=-1
    "probability": [True],  # [True, False], default=False
    "random_state": [RANDOM_STATE],  # int, RandomState instance or None, default=None
    "shrinking": [True],  # [True, False], default=True
    "tol": [0.1],  # float, default=1e-3
    "verbose": [True],  # [True, False], default=False
}
grid_model_nu_svc_sigmoid_kernel = train_model(model_class=NuSVC(), name="NuSVC with Grid hyperparameters", grid_params=grid_params_nu_svc_sigmoid_kernel)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: NuSVC with Grid hyperparameters
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Best params:  {'break_ties': False, 'cache_size': 200, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'sigmoid', 'max_iter': -1, 'nu': 0.6, 'probability': True, 'random_state': 1729, 'shrinking': True, 'tol': 0.1, 'verbose': True}
Best score:  0.9185546334716459
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.87      0.82      0.85        40
           2       0.86      0.95      0.90        39
           3       0.97      0.80      0.88        40
           4       1.00      0.93      0.96        40
           5       0.95      0.97      0.96        40
           6       0.98      1.00      0.99        40
           7       0.85      1.00      0.92        39
           8       0.97

## Порівняння результатів

In [ ]:
value_results(name="SVC with default hyperparameters", predict=model_svc_default.predict(X=x_test))
value_results(name="SVC with linear kernel", predict=grid_model_svc_linear_kernel.predict(X=x_test))
value_results(name="SVC with poly kernel", predict=grid_model_svc_poly_kernel.predict(X=x_test))
value_results(name="SVC with rbf kernel", predict=grid_model_svc_rbf_kernel.predict(X=x_test))
value_results(name="SVC with sigmoid kernel", predict=grid_model_svc_sigmoid_kernel.predict(X=x_test))

value_results(name="LinearSVC default ", predict=model_linear_svc_default.predict(X=x_test))
value_results(name="LinearSVC with Grid hyperparameters ", predict=grid_model_linear_svc.predict(X=x_test))

value_results(name="NuSVC default ", predict=model_nu_svc_default.predict(X=x_test))
value_results(name="NuSVC with linear kernel", predict=grid_model_nu_svc_linear_kernel.predict(X=x_test))
value_results(name="NuSVC with poly kernel", predict=grid_model_nu_svc_poly_kernel.predict(X=x_test))
value_results(name="NuSVC with rbf kernel", predict=grid_model_nu_svc_rbf_kernel.predict(X=x_test))
value_results(name="NuSVC with sigmoid kernel", predict=grid_model_nu_svc_sigmoid_kernel.predict(X=x_test))

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with default hyperparameters
Accuracy: 0.9644670050761421
Precision: 0.9644670050761421
Recall: 0.9644670050761421
F1 Score: 0.9644670050761421
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with linear kernel
Accuracy: 0.9746192893401016
Precision: 0.9746192893401016
Recall: 0.9746192893401016
F1 Score: 0.9746192893401016
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with poly kernel
Accuracy: 0.9898477157360406
Precision: 0.9898477157360406
Recall: 0.9898477157360406
F1 Score: 0.9898477157360406
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
MODEL: SVC with rbf kernel
Accuracy: 0.9746192893401016
Precision: 0.9746192893401016
Recall: 0.9746192893401016
F1 Score: 0.97461

# Висновок

Провів тести на 3 різних моделях (SVC, NuSVC, LinearSVC) з дефолтними, та підібраними гіперпараметрами для досягнення найкращого предикту.

**Балансування** класів не робив, тому що класи майже сбалансовані, максимальна різниця становить ~6% від кількості екземплярів.

**Масштабування** проводив на трьох скалєрах: StandardScaler, RobustScaler, PowerTransformer. Найкращий рещультат показав StandardScaler.

**Feature Engineering** по тз пропускаємо.

Найкращі та стабільні результати показали 2 моделі:
1. SVC() модель з мануально підібраними гіперпараметрами, та ядром "poly"
2. NuSVC() модель з мануально підібраними гіперпараметрами, та ядром "poly"

Обидві моделі видають F1 Score ~ 0.9898, якщо округлити: 0.99

Варто відмітити, що моделі "SVC" та "NuSVC" - це майже одна й та сама модель, за вийнятком двох гіперпараметрів: "nu" у "NuSVC", та "C" у "SVC". Тому логічно, що у них однаковий предикт.

Також, як можна побачити по метриках, майже завжди кращий результат показують моделі з "підігнаними" гіперпараметрами (за вийнятком ядра "sigmoid", з ним предикт вище ніж ніж з дефолтними гіперпараметрами не вдалося розігнати), так як дефолтні не можуть завжди підходити до будь-якого датасету.

Варто завжди приділяти цьому увагу.